In [2]:
import numpy as np
import numpy.linalg as la
import matplotlib.pyplot as plt
import io
from L101_utils.data_paths import wikift, bolu_gender_specific, bolu_equalize_pairs, googlew2v, bolu_definitional_pairs, googlew2vtxt
from L101_utils.mock_model import MockModel
from copy import deepcopy

In [3]:
emb = MockModel.from_file(googlew2v, mock=False)

['c:\\users\\user\\desktop\\coursework\\nlp\\l101_bias_space_study\\data\\GoogleNews-vectors-negative300'] bin


In [4]:
emb.vectors = emb.vectors / np.linalg.norm(emb.vectors,  axis=1)[..., None]

In [7]:
import json


def get_pc_projection_boluk(X, k=1, mean_rev=True):
    
    mean_rev = int(mean_rev)
    n, d = X.shape
#     import pdb; pdb.set_trace()
    X = X - mean_rev * X.mean(axis=0)
    C = (X.T.dot(X) / n)
    D, V = la.eigh(C)
    V = V[:, :k]
    return V.dot(V.T), V


def create_nu_boluk(X, P):
    mu = X.mean(axis=0)
    nu = mu - P.dot(mu)
    return nu, P.dot(mu)


def equalize_boluk(E, P, N=None, debug=True, test_vec=None):
    nu, mu_b = create_nu_boluk(E, P)
#     nu =  (np.eye(len(P)) - P).dot( E.mean(axis=0) )
    if debug:
        print(np.linalg.norm(E, axis=1))
    
    Eo = E.copy()
    E = (E - E.mean(axis=0)[None, ...]).dot(P)
    if debug:
        print(np.linalg.norm(E, axis=1))
#         plt.imshow(np.abs(P))
        print(np.min(P.ravel()), np.max(P.ravel()))
#     print(np.sign(E)[:,1:7])
    E /= np.linalg.norm(E, axis=1)[..., None]
#     print(np.sign(E)[:,1:7])
#     import pdb;pdb.set_trace()?@|?
    
#     E = E.dot(P)
    
    v = np.linalg.norm(nu)
    fac = np.sqrt(1 - v**2)
    remb = nu +  fac * E
    
    if debug:
        print(np.linalg.norm(E,axis=1))
        print(np.linalg.norm(remb, axis=1))
        print(fac)
#     print(np.sign(E)[:,1:7], "lasteus")
    return remb, E


def neutralise_boluk(X, P):
    print("start neutralise")
    I = np.eye(P.shape[0])
    out =  X.dot( (I - P).T )
    print("done matrix mult (neutralise)", out.shape)
    return out


def generate_subspace_projection(emb,
                                 pair_file=bolu_definitional_pairs,
                                 n_components=1):
    with open(pair_file, "r") as f:
        pairs = json.load(f)
    
    matrix = []
    for a, b in pairs:
        center = (emb.vectors[emb.vocab[a.lower()].index] + emb.vectors[emb.vocab[b.lower()].index])/2
#         import pdb; pdb.set_trace()
        matrix.append(emb.vectors[emb.vocab[a.lower()].index] - center)
        matrix.append(emb.vectors[emb.vocab[b.lower()].index] - center)
        
    matrix = np.asarray(matrix)
#     import pdb; pdb.set_trace()
    P, V = get_pc_projection_boluk(matrix, k=n_components)
    assert (P == P.T).all()
#     import pdb; pdb.set_trace()
    return P, V


def hard_debiase(emb,
                 gender_specific_file=bolu_gender_specific,
                 equalize_pair_file=bolu_equalize_pairs,
                 def_pair_file=bolu_definitional_pairs,
                 n_components=1):
    
#     emb = deepcopy(emb)
    
    print("projection started")
    P, V = generate_subspace_projection(emb, def_pair_file, n_components)
    print("projection done")
    
    with open(gender_specific_file, "r") as f:
        gendered_words = set(json.load(f))
    
    print("neutralisation started")
    all_words = set(emb.vocab.keys())
    neutral_words = list(all_words - gendered_words)
    print("getting indices should be fast")
    indices = [emb.vocab[k].index for k in neutral_words[0:10000]]
    print("created neutral word set", len(neutral_words))
    neutral = emb.vectors[indices,:]
    print("done indexing into neutral embs")
    test = neutralise_boluk(neutral, P)
    emb.vectors[indices,:] = test
    print("neutralisation done")
    
    with open(bolu_equalize_pairs, "r") as f:
        equalize_words = json.load(f)
    
    candidates = {x for e1, e2 in equalize_words for x in [(e1.lower(), e2.lower()),
                                                           (e1.title(), e2.title()),
                                                           (e1.upper(), e2.upper())]}
    print(candidates, "started equalising")
#     import pdb; pdb.set_trace()
    for (e1, e2) in candidates:
        if (e1 in all_words and e2 in all_words):
            word2index  = [emb.vocab[e1].index, emb.vocab[e2].index]
            E = emb.vectors[word2index,:]
            remb, _ = equalize_boluk(E, P, debug=False, test_vec=V)
#             print("remb", test[0].dot(remb.T))
#             print(np.sign(remb)[:, 0:10])
#             import pdb; pdb.set_trace()

#             print(np.linalg.norm(_, axis=1))
#             print( np.linalg.norm((_[0,:]) + (_[1,:])) )
            print(e1,e2)
            print( (np.sign(_[0,:]) == -np.sign(_[1,:])).all() ) 
#             print(np.sign(_)[:,1:7])
#             print(V.T.dot(E[0,:]), V.T.dot(E[1,:]) )
           
            emb.vectors[word2index,:] = remb
    return emb

In [8]:
embnasius = hard_debiase(emb)


projection started
projection done
neutralisation started
getting indices should be fast
created neutral word set 2998559
done indexing into neutral embs
start neutralise
done matrix mult (neutralise) (10000, 300)
neutralisation done
{('His', 'Her'), ('Brothers', 'Sisters'), ('grandson', 'granddaughter'), ('Gelding', 'Mare'), ('BUSINESSMAN', 'BUSINESSWOMAN'), ('grandfather', 'grandmother'), ('Fraternity', 'Sorority'), ('Catholic_Priest', 'Nun'), ('testosterone', 'estrogen'), ('WIVES', 'HUSBANDS'), ('king', 'queen'), ('Councilman', 'Councilwoman'), ('Boys', 'Girls'), ('prince', 'princess'), ('Males', 'Females'), ('wives', 'husbands'), ('GELDING', 'MARE'), ('gelding', 'mare'), ('He', 'She'), ('Dad', 'Mom'), ('MEN', 'WOMEN'), ('UNCLE', 'AUNT'), ('BROTHERS', 'SISTERS'), ('fella', 'granny'), ('EX_GIRLFRIEND', 'EX_BOYFRIEND'), ('his', 'her'), ('monastery', 'convent'), ('BOY', 'GIRL'), ('PRINCE', 'PRINCESS'), ('Uncle', 'Aunt'), ('SONS', 'DAUGHTERS'), ('chairman', 'chairwoman'), ('Grandson', '

In [ ]:
emb[set(["hello", "punk"])].shape

In [ ]:
# Auxiliary functions


def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for i, line in enumerate(fin):
        if i == 100000: break
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = np.asarray(list(map(float, tokens[1:])))
        data[tokens[0]] /=  np.linalg.norm(data[tokens[0]]) 
    return data


X = load_vectors(wikift);

In [ ]:
biased_dat = """he, her, his, she, him, man, women, men, woman, spokesman, wife, himself, son, mother, father, chairman,
daughter, husband, guy, girls, girl, boy, boys, brother, spokeswoman, female, sister, male, herself, brothers, dad,
actress, mom, sons, girlfriend, daughters, lady, boyfriend, sisters, mothers, king, businessman, grandmother,
grandfather, deer, ladies, uncle, males, congressman, grandson, bull, queen, businessmen, wives, widow,
nephew, bride, females, aunt, prostate cancer, lesbian, chairwoman, fathers, moms, maiden, granddaughter,
younger brother, lads, lion, gentleman, fraternity, bachelor, niece, bulls, husbands, prince, colt, salesman, hers,
dude, beard, filly, princess, lesbians, councilman, actresses, gentlemen, stepfather, monks, ex girlfriend, lad,
sperm, testosterone, nephews, maid, daddy, mare, fiance, fiancee, kings, dads, waitress, maternal, heroine,
nieces, girlfriends, sir, stud, mistress, lions, estranged wife, womb, grandma, maternity, estrogen, ex boyfriend,
widows, gelding, diva, teenage girls, nuns, czar, ovarian cancer, countrymen, teenage girl, penis, bloke, nun,
brides, housewife, spokesmen, suitors, menopause, monastery, motherhood, brethren, stepmother, prostate,
hostess, twin brother, schoolboy, brotherhood, fillies, stepson, congresswoman, uncles, witch, monk, viagra,
paternity, suitor, sorority, macho, businesswoman, eldest son, gal, statesman, schoolgirl, fathered, goddess,
hubby, stepdaughter, blokes, dudes, strongman, uterus, grandsons, studs, mama, godfather, hens, hen, mommy,
estranged husband, elder brother, boyhood, baritone, grandmothers, grandpa, boyfriends, feminism, countryman,
stallion, heiress, queens, witches, aunts, semen, fella, granddaughters, chap, widower, salesmen, convent,
vagina, beau, beards, handyman, twin sister, maids, gals, housewives, horsemen, obstetrics, fatherhood,
councilwoman, princes, matriarch, colts, ma, fraternities, pa, fellas, councilmen, dowry, barbershop, fraternal,
ballerina"""

biased_dat = biased_dat.replace(" ","").replace("\n","").strip("\r").split(",")

In [ ]:
G = [X[k]  for k in biased_dat if k in X]
N = [X[k]  for k in set(X.keys()) - set(biased_dat) ]

In [ ]:
G = np.array(G)
N = np.array(N)

In [ ]:
P = get_pc_projection(G)

In [ ]:
Nv = neutralise(N, P)

In [ ]:
Nv /= np.linalg.norm(Nv, axis=1)[..., None]

In [ ]:
def get_pc_projection(X, k=1):
    n, d = X.shape
    X = X - X.mean(axis=0)
    C = (X.T.dot(X) / n)
    D, V = la.eigh(C)
    V = V[:, :k]
    return V.dot(V.T)

def create_nu(X, P):
    mu = X.mean(axis=0)
    nu = mu - P.dot(mu)
    return nu, P.dot(mu)

def equalize(E, P, N=None, debug=True):
    
    nu, mu_b = create_nu(E, P)
#     print(E[0,:] == E[1,:])
    if debug:
        print(np.linalg.norm(E, axis=1))
    
    E = E.dot(P) - 0 * mu_b
    
    if debug:
        print(np.linalg.norm(E, axis=1))
        plt.imshow(np.abs(P))
        print(np.min(P.ravel()), np.max(P.ravel()))

    E /= np.linalg.norm(E, axis=1)[..., None]
    
    v = np.linalg.norm(nu)
    fac = np.sqrt(1 - v**2)
    remb = nu +  fac * E
    
    if debug:
        print(np.linalg.norm(E,axis=1))
        print(np.linalg.norm(remb, axis=1))
    
    return remb, E

def neutralise(X, P):
    
    return X.dot((np.eye(P.shape[0]) - P))

# Make equaliser set out of : he, his, her, she, him, man, women, men, woman
E = G[0:9, :]

Eveq, E = equalize(E, P, Nv)

In [ ]:
Eveq

In [ ]:
Eveq.dot(Nv[-1,:])

In [ ]:
np.linalg.norm(Eveq, axis=1)

In [ ]:
for i in range(9):
    print((np.sign(Eveq[0,:]) == np.sign(Eveq[i,:]) ).all())

In [ ]:
r = P.dot(E[0,:].T)
n = (np.eye(P.shape[0]) - P).dot(r)

In [ ]:
r.dot(n)

In [ ]:
plt.imshow(np.abs(P))


In [ ]:
(P == P.T).all()

In [ ]:
np.diag(P).max(), np.diag(P).min(), (P - np.diag(np.diag(P))).max(), np.diag(np.abs(P)).max(), np.min(P), np.max(P), np.std(P)

In [ ]:
n, d = G.shape
Gc  = G - G.mean(axis=0)
C = (Gc.T.dot(Gc) / n)

In [ ]:
plt.imshow(np.abs(C))

In [ ]:
np.linalg.matrix_rank(C)